## 1. Example of Minmax Featurization


In [75]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin ## inheritance..
class Minmaxreg(BaseEstimator,TransformerMixin):
    def __init__(self,a) -> None:
        self.a=a
    def fit(self,X):
        self.X_max=np.max(X)
        self.X_min=np.min(X)
        return self
    def transform(self,X):
        
        try:
            getattr(self, "X_min") or getattr(self,"X_max")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")
        X=X.copy()
        X=(X-self.X_min)/(self.X_max-self.X_min)
        return X
    

## Output


In [76]:
X_train=[1,2,3,4,5,6,7,8,9,10]
X_test=[1,3,5]
temp=4
scale=Minmaxreg(temp)
scale.fit(X_train)
print("X_train_reg: {}".format(scale.transform(X_train))) ## 결과 1

scale.fit(X_test)
print("X_test_reg: {}".format(scale.transform(X_test))) ## 결과 2

X_train_reg: [0.         0.11111111 0.22222222 0.33333333 0.44444444 0.55555556
 0.66666667 0.77777778 0.88888889 1.        ]
X_test_reg: [0.  0.5 1. ]


In [77]:

print("X_train_reg: {}".format(scale.fit_transform(X_train))) ## 결과 1
print("X_test_reg: {}".format(scale.transform(X_test))) ## 결과 2

X_train_reg: [0.         0.11111111 0.22222222 0.33333333 0.44444444 0.55555556
 0.66666667 0.77777778 0.88888889 1.        ]
X_test_reg: [0.         0.22222222 0.44444444]


In [78]:
scale.get_params()
scale.set_params(**{'a':1653245})

Minmaxreg(a=1653245)

## 2. Ridge Rigression class

아래코드는 본강의 의 HW-2 에있는 ridge_regression.py를 그대로 작성한 것입니다.

In [99]:
from sklearn.base import BaseEstimator, RegressorMixin
from scipy.optimize import minimize
from setup_problem import *
import numpy as np
import pandas as pd
from gradient import *
class RidgeRegression(BaseEstimator, RegressorMixin):
    """ ridge regression"""

    def __init__(self, l2reg=1):
        if l2reg < 0:
            raise ValueError('Regularization penalty should be at least 0.')
        self.l2reg = l2reg

    def fit(self, X, y=None):
        n, num_ftrs = X.shape
        # convert y to 1-dim array, in case we're given a column vector
        y = y.values.reshape(-1,1)
        def ridge_obj(w):
            predictions = np.dot(X,w)
            residual = y - predictions
            empirical_risk = np.sum(residual**2) / n
            l2_norm_squared = np.sum(w**2)
            objective = empirical_risk + self.l2reg * l2_norm_squared
            return objective
        self.ridge_obj_ = ridge_obj

        w_0 = np.zeros(num_ftrs)
        self.w_ = minimize(ridge_obj, w_0).x
        return self

    def predict(self, X, y=None):
        try:
            getattr(self, "w_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")
        return np.dot(X, self.w_)

    def score(self, X, y):
        # Average square error
        try:
            getattr(self, "w_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")
        residuals = self.predict(X) - y
        return np.dot(residuals, residuals)/len(y)
    

이전 Gradient Descent 를 구현 했을때, 사용했던 데이터를 그대로 이용하였습니다.

In [120]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas

df=pandas.read_csv("data.csv") ## Jupiter Notebook이라 간단하게 표현가능, 실제로는 경로를 더 자세하게 표시해줘야함.
y=df['y']
X=df.loc[:,df.columns!='y']
X_train,X_test,y_train,y_test=splitData(X,y)
## 성능 평가를 위해 test set 과 train set을 분리.
X_train,X_test=featurescale(X_train,X_test)
ridge=RidgeRegression(l2reg=0.1)
print(ridge.fit(X_train,y_train).predict(X_train))
print(y_test)

[0.00475374 0.0047071  0.00482957 0.00479079 0.0050229  0.00479994
 0.00465231 0.00496294 0.00504756 0.00511747 0.00480354 0.00447414
 0.0049363  0.00502942 0.00464461 0.00501888 0.00476037 0.00460954
 0.004786   0.00491225 0.00352897 0.00515757 0.00472208 0.00487714
 0.00502122 0.00460349 0.00489362 0.00474189 0.00497075 0.00516706
 0.00486863 0.00479226 0.00494532 0.00494294 0.0047825  0.00478913
 0.00462048 0.00356574 0.00477012 0.00486362 0.00354656 0.00486926
 0.00482588 0.00522112 0.00474458 0.00515039 0.00358054 0.00480658
 0.0049779  0.00451915 0.00465096 0.00352779 0.00495659 0.00501862
 0.00462391 0.00507063 0.00485641 0.0046842  0.00504393 0.00477464
 0.00488952 0.00470381 0.00506592 0.00506863 0.00474172 0.0049091
 0.00475167 0.0045377  0.00491403 0.00482927 0.00469954 0.00473265
 0.00456736 0.00504039 0.00466602 0.00450052 0.00353932 0.00529552
 0.0049423  0.00496112 0.00471978 0.00512093 0.00483424 0.00475041
 0.00474181 0.00495659 0.00356711 0.00490377 0.00510273 0.00449